# HW3

## Text Processing

### Q1

1. Modify the code I wrote in lecture 8 with what you have learnt in lecture 9 and correctly tokenize the text both on the word and sentence level, and by removing the stopwords. Rewrite the `getSummary` function and all the other functions that it depends by maing these corrections.

2. Rewrite the code I wrote for `getKeywords` function making the same corrections.

3. Test your code from parts 1 and 2 on random articles from the Guardian.

4. Rewrite the `getSubjectGuardian` function for another newspaper in English, and test your code from part 1 and 2 on random articles from this new newspaper.

In [1]:
import requests
import nltk
import regex as re
import spacy
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
from bs4 import BeautifulSoup
from xmltodict import parse
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA

In [2]:
with requests.get('https://www.theguardian.com/technology/rss') as link:
    raw = parse(link.text)

In [3]:
nal = raw['rss']['channel']['item']

In [4]:
def getText(url):
    with requests.get(url) as link:
        raw = BeautifulSoup(link.content,'html.parser')
    return ' '.join([x.text for x in raw.find_all('p')])

In [5]:
text=getText(nal[0]['link'])

In [6]:
omats = {'sentences': sent_tokenize(text),
         'words': word_tokenize(text)}

In [7]:
omats.update({'sentences': [re.sub(r'[^\p{Letter}\s]','',sentence.lower()) for sentence in omats['sentences']],
              'words': [re.sub(r'[^\p{Letter}]','',word.lower()) for word in omats['words']]})

In [8]:
swEN = set(stopwords.words('english'))
swEN

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [9]:
reduced = [word for word in omats['words'] if word not in swEN]
reduced

['world',
 '',
 'richest',
 'person',
 'aims',
 'reassure',
 'followers',
 '',
 'saying',
 'seek',
 'politicise',
 'social',
 'media',
 'network',
 'elon',
 'musk',
 'said',
 'twitter',
 'must',
 '',
 'politically',
 'neutral',
 '',
 'comment',
 'posted',
 'last',
 'night',
 'wave',
 'account',
 'deletions',
 'leftleaning',
 'users',
 'social',
 'media',
 'network',
 '',
 'days',
 'since',
 'musk',
 '',
 '',
 'bn',
 '',
 'bn',
 '',
 'acquisition',
 'offer',
 'accepted',
 'twitter',
 '',
 'board',
 '',
 'hundreds',
 'thousands',
 'users',
 'closed',
 'accounts',
 'site',
 '',
 'company',
 'confirmed',
 '',
 'leading',
 'dip',
 'follower',
 'numbers',
 'leftleaning',
 'politicians',
 'celebrities',
 'barack',
 'michelle',
 'obama',
 '',
 'taylor',
 'swift',
 'jeremy',
 'corbyn',
 '',
 'meanwhile',
 '',
 'rightwing',
 'influencers',
 'farright',
 'congresswoman',
 'marjorie',
 'taylor',
 'greene',
 '',
 'boris',
 'johnson',
 'ted',
 'cruz',
 'large',
 'gains',
 'new',
 'users',
 'sign',
 

In [10]:
def processText(text):
    sentences = re.split(r'[.?!]',text)
    return [re.sub(r'[^\w\s]','',x.lower()) for x in sentences]

In [11]:
def getMatrix(sentences):
    vectorizer = CountVectorizer()
    return vectorizer.fit_transform(sentences)

In [12]:
def getSummary(text,k):
    sentences = processText(text)
    matrix = getMatrix(sentences)
    projection = PCA(n_components=1)
    weights = projection.fit_transform(matrix.toarray())
    res = list(zip(weights.transpose()[0],range(112),sentences))
    tmp = sorted(res,key=lambda x: x[0],reverse=True)[:k]
    return sorted(tmp, key=lambda x: x[1])

In [13]:
getSummary(text,6)

[(4.516465690943847,
  10,
  ' the former president donald trump was banned from the social network in 2021 for his role in encouraging the storming of the us capitol while greene had her personal account permanently suspended after breaking the platforms fivestrike rule'),
 (4.474975719233619,
  16,
  ' katy perry the pop star who is the sites third biggest user lost 7000 followers in a few days while obama whose 132 million followers make the former us presidents account the most popular on the site lost 5000 on tuesday alone'),
 (1.9216698829643972,
  18,
  ' cruz the junior senator for texas added more than 60000 followers while greene gained more than 100000 followers in the last week a tenfold increase in the normal rate'),
 (3.893372354309854,
  20,
  ' some speculated that the changes were the result of action from twitter cleaning house in preparation for the acquisition but the social network said they were the result of organic activity'),
 (2.5005054706430534,
  29,
  ' the

### Q2

Write a function that returns all named entities (proper names, country names, corporation names only) from a URL. Function should take the URL as the input and must return the list of named entities from that URL. Test your code on random articles from the Guardian. Don't use the NLTK's NER that I demonstrated during the lecture. Use the SpaCY's NER function.

In [14]:
NER = spacy.load("en_core_web_sm")

In [15]:
def function(url):
    with requests.get(url) as link:
        raw = BeautifulSoup(link.content,'html.parser')
    txt=' '.join([x.text for x in raw.find_all('p')])
    res = NER(txt)
    person=[]
    country=[]
    corp=[]
    for word in res.ents:
        if word.label_== 'PERSON':
            person.append(word.text)
        if word.label_== 'GPE':
            country.append(word.text)
        if word.label_== 'ORG':
            corp.append(word.text)
    print('proper person names' ,person)     
    print('country',country)
    print('corparation',corp)

In [16]:
function(nal[0]['link'])

proper person names ['Musk', 'Barack', 'Michelle Obama', 'Taylor Swift', 'Jeremy Corbyn', 'Marjorie Taylor Greene', 'Boris Johnson', 'Ted Cruz', 'Musk', 'Musk', 'Musk', 'Donald Trump', 'Musk', 'Ben Shapiro', 'Katy Perry', 'Michelle Obama', 'Cruz', 'Johnson', 'Jeremy Corbyn', 'Musk', 'Dick Costolo', 'Musk', 'Musk', 'Musk', 'Musk']
country ['US', 'US', '~80', 'US', 'Texas']
corparation ['Twitter', 'Twitter', 'Twitter', 'Greene', 'Greene', 'Twitter', 'Labour', 'Twitter', 'Vijaya Gadde', 'Twitter', 'Twitter', 'the Federal Trade Commission', 'Reuters', 'Information', 'Twitter']


### Q3

1. Write a function that returns the most positive and the most negative sentences from a text. The function must take the text as the input and must return a 2-tuple: the first element as the most positive and the second as the most negative sentence with their polarity scores.

2. Test your function on random articles from the Guardian.

In [17]:
analyzer = SentimentIntensityAnalyzer()

In [18]:
def finds(text):
    sentences = sent_tokenize(text)
    a=[]
    for x in sentences:
        a.append(analyzer.polarity_scores(x))
    pozitif=[]
    negatif=[]
    for i in a:
        pozitif.append(i['pos'])
        negatif.append(i['neg'])
    indexp=pozitif.index(max(pozitif))
    indexn=negatif.index(max(negatif))
    for i in sentences:
        if analyzer.polarity_scores(i)==a[indexp]:
            poz=i
        if analyzer.polarity_scores(i)==a[indexn]:
            neg=i
    return poz ,neg

In [19]:
finds(text)

('In earlier comments, Musk has been outspoken about his desire to promote free speech on Twitter, saying that he is “against censorship that goes far beyond the law”.',
 'His tweets sparked tens of thousands of abusive messages targeted at the executive, and a public rebuke from a former Twitter chief executive, Dick Costolo.')